# Data Preprocessing Tools

## Importing the libraries

In [41]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Importing the dataset

In [42]:
# Load Excel files
df = pd.read_csv("0_service_now_raw_data/service_now_only.csv")

In [43]:
print(df.columns)

Index(['Asset Number', 'Incident_Number', 'Short_Description', 'Status',
       'Priority', 'Category', 'Subcategory', 'Assigned_To', 'Assigned_Group',
       'Created_Date'],
      dtype='object')


In [44]:
# Selective renaming for df
df_rename = df.rename(columns={
    'Asset Number': 'Asset_Number'
})


In [45]:
print(df_rename.columns)

Index(['Asset_Number', 'Incident_Number', 'Short_Description', 'Status',
       'Priority', 'Category', 'Subcategory', 'Assigned_To', 'Assigned_Group',
       'Created_Date'],
      dtype='object')


In [46]:
# Your selected columns
req_cols = ['Asset_Number', 'Created_Date']

# Load only required columns
df = df_rename[req_cols]


In [47]:
print(df.head())

   Asset_Number         Created_Date
0        100080  2024 04 09T15:58:22
1        100081  2024 04 17T08:05:22
2        300006  2024 04 10T08:33:47
3        300007  2024 04 05T14:22:33
4        300008  2024 04 11T11:45:33


## Custom Data Creation

In [48]:
from datetime import timedelta
import random

# Convert to datetime.date (date only, no time)
df.loc[:, "Created_Date"] = pd.to_datetime(df["Created_Date"], format="%Y %m %dT%H:%M:%S").dt.date

# Initialize list to hold simulated data
simulated_data = []

# For each unique asset
for _, row in df.iterrows():
    asset_id = row["Asset_Number"]
    created_date = row["Created_Date"]

    # Convert start_date to datetime.date as well
    start_date = (pd.to_datetime(created_date) - pd.DateOffset(months=36)).date()

    # Generate random number of incidents
    num_incidents = random.randint(300, 350)

    # Generate n-1 random incident dates before the Created_Date
    incident_dates = [
        start_date + timedelta(days=random.randint(0, (created_date - start_date).days - 1))
        for _ in range(num_incidents - 1)
    ]

    # Add the actual Created_Date as the latest incident
    incident_dates.append(created_date)

    # Sort the list of incident dates
    incident_dates.sort()

    # Append to the final list
    for date in incident_dates:
        simulated_data.append({"Asset_Number": asset_id, "Incident_Date": date})

# Create the final DataFrame
incident_df = pd.DataFrame(simulated_data)

# Convert Incident_Date to datetime (for diff)
incident_df["Incident_Date"] = pd.to_datetime(incident_df["Incident_Date"])

# Sort and calculate gap in days
incident_df = incident_df.sort_values(["Asset_Number", "Incident_Date"])
incident_df["Gap_Days"] = incident_df.groupby("Asset_Number")["Incident_Date"].diff().dt.days

incident_df.reset_index(drop=True, inplace=True)



## Export Updated Excel

In [ ]:
# Create the folder if it doesn't exist
os.makedirs("1_service_now_historic_time_series_data", exist_ok=True)

# Save the Excel file inside the folder
incident_df.to_csv("1_service_now_historic_time_series_data/1_service_now_historic_time_series_data.csv", index=False)